In [2]:
#! pip install langchain replicate

In [3]:
#load API key
def read_api_key(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

REPLICATE_API_TOKE = read_api_key('../API_Key/REPLICATE_API_TOKEN.txt')
OPENAI_API_KEY = read_api_key('../API_Key/OPENAI_API_KEY.txt')


In [4]:
import os
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKE
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [5]:
# Set up Local llama2
from langchain.chat_models import ChatOllama

llama2_chat = ChatOllama(model="llama2:13b-chat")
llama2_code = ChatOllama(model="codellama:7b-instruct")

# API
from langchain.llms import Replicate

# os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
replicate_id = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
llama2_chat_replicate = Replicate(
    model=replicate_id, model_kwargs={"temperature": 0.01, "max_length": 500, "top_p": 1}
)

In [6]:
llm = llama2_chat

In [7]:
# #Set MySQL and SQLite DB
# from typing import Any, Dict, List, Tuple, Union
# from langchain.utilities import SQLDatabase
# import pymysql
# KV = Dict[str, Any]
# Query = Tuple[str, List]

# class mySQL_DB:
#     def __init__(self, host: str, port: int, user: str, password: str, database: str):
#         conn = pymysql.connect(
#             host=host,
#             port=port,
#             user=user,
#             password=password,
#             database=database,
#             cursorclass=pymysql.cursors.DictCursor,
#         )
#         self.conn = conn
#         self.database = database

#     def get_cursor(self):
#         return self.conn.cursor()
    
#     def run_query(self, query: str, args: List, ret_result: bool) -> Union[List[KV], int]:

#         cur = self.get_cursor()
#         count = cur.execute(query, args=args)
#         if ret_result:
#             return cur.fetchall()
#         else:
#             return count
    
#     def get_schema(self):
#         """
#         Fetches all tables and their columns from a specified MySQL database.
#         :return: Dictionary with table names as keys and list of columns as values
#         """
#         connection = self.conn
#         cursor = connection.cursor()

#         # Fetch tables and columns
#         query = """
#         SELECT 
#             TABLE_NAME, 
#             COLUMN_NAME 
#         FROM 
#             information_schema.COLUMNS 
#         WHERE 
#             TABLE_SCHEMA = %s
#         ORDER BY 
#             TABLE_NAME, 
#             ORDINAL_POSITION;
#         """
#         cursor.execute(query, (self.database,))

#         # Process the results
#         tables_columns = {}
#         for table_name, column_name in cursor:
#             if table_name not in tables_columns:
#                 tables_columns[table_name] = []
#             tables_columns[table_name].append(column_name)
#         return tables_columns
    
# class SQLite_DB:
#     def __init__(self, db_path: str):
#         self.db = SQLDatabase.from_uri(db_path, sample_rows_in_table_info=0)
    
#     def get_schema(self):
#         return self.db.get_table_info()
    
#     def run_query(self,query):
#         return self.db.run(query)
    
    

In [8]:
    #Set up SQLite DB using the above classes
from langchain.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///./Chinook.db", sample_rows_in_table_info=0)
# db = SQLite_DB("sqlite:///./Chinook.db")    

In [9]:
def get_schema(_):
    return db.get_table_info()
def run_query(query):
    print("running query\n", query)
    return db.run(query)

In [10]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)


prompt_llama2 = ChatPromptTemplate.from_messages(
    [
        ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
        ("human", template),
    ]
)


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [12]:
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) FROM Employee'

In [13]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [14]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: run_query(x["query"]),
    )
    | prompt_response
    | model
)

In [15]:
full_chain.invoke({"question": "How many employees are there?"})

running query
 SELECT COUNT(*) FROM Employee


AIMessage(content='There are 8 employees.')

In [16]:
full_chain.invoke({"question": "What are the top 3 best-selling artists from the database?"})

running query
 SELECT a.Name AS Artist, COUNT(il.InvoiceLineId) AS TotalSales
FROM Artist a
JOIN Album al ON a.ArtistId = al.ArtistId
JOIN Track t ON al.AlbumId = t.AlbumId
JOIN InvoiceLine il ON t.TrackId = il.TrackId
GROUP BY a.Name
ORDER BY TotalSales DESC
LIMIT 3;


AIMessage(content='The top 3 best-selling artists from the database are Iron Maiden with 140 total sales, U2 with 107 total sales, and Metallica with 91 total sales.')

In [17]:
#Now using llama2_chat


sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)
db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info=0)

In [29]:
# Prompt
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

template = """Given an input question, convert it to a SQL query. No pre-amble. Based on the table schema below, write a SQL query that would answer the user's question:
{schema}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

# Chain to query with memory
from langchain_core.runnables import RunnableLambda

sql_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)


def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)
    return output["output"]


sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save
sql_response_memory.invoke({"question": "What team is Klay Thompson on?"})

"\nSELECT Team\nFROM nba_roster\nWHERE NAME = 'Klay Thompson';"

In [30]:
print(memory.load_memory_variables({"question": "What team is Klay Thompson on?"}))

{'history': [HumanMessage(content='What team is Klay Thompson on?'), AIMessage(content="\nSELECT Team\nFROM nba_roster\nWHERE NAME = 'Klay Thompson';")]}


In [27]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

full_chain = (
    RunnablePassthrough.assign(query=sql_response_memory)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: run_query(x["query"]),
    )
    | prompt_response
    | llm
)

full_chain.invoke({"question": "What are the other members in his team?"})

running query
 
SELECT NAME, Jersey FROM nba_roster WHERE Team = (SELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson');


AIMessage(content="\nOther members of Klay Thompson's team are:\n\n* Stephen Curry (#30)\n* Draymond Green (#23)\n* JaMychal Green (#1)\n* Andre Iguodala (#9)\n* Trayce Jackson-Davis (NA)\n* Cory Joseph (#18)\n* Jonathan Kuminga (00)\n* Anthony Lamb (#40)\n* Kevon Looney (#5)\n* Moses Moody (#4)\n* Chris Paul (NA)\n* Gary Payton II (#8)\n* Brandin Podziemski (NA)\n* Lester Quinones (#25)\n* Dario Saric (#9)")

In [23]:
print(memory.load_memory_variables({"question": "Who else are in his team?"}))
print(sql_response_memory)

{'history': [HumanMessage(content='What team is Klay Thompson on?'), AIMessage(content="\nSELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson';"), HumanMessage(content='What is his salary?'), AIMessage(content="\nSELECT SALARY FROM nba_roster WHERE NAME = 'Klay Thompson';")]}
first=RunnableAssign(mapper={
  output: RunnableAssign(mapper={
            schema: RunnableLambda(get_schema),
            history: RunnableLambda(lambda x: memory.load_memory_variables(x)['history'])
          })
          | ChatPromptTemplate(input_variables=['history', 'question', 'schema'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['schema'], template="Given an i

In [22]:

memory = ConversationBufferMemory(return_messages=True)
template = """Given an input question, convert it to a SQL query. No pre-amble. Based on the table schema below, write a SQL query that would answer the user's question:
{schema}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)



sql_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)
 

def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)
    return output["output"]


sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save
sql_response_memory.invoke({"question": "What team is Klay Thompson on?"})

# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

full_chain = (
    RunnablePassthrough.assign(query=sql_response_memory)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: run_query(x["query"]),
    )
    | prompt_response
    | llm
)

full_chain.invoke({"question": "What is his salary?"})

running query
 
SELECT SALARY FROM nba_roster WHERE NAME = 'Klay Thompson';


AIMessage(content="Natural language response:\n\nKlay Thompson's salary is $43,219,440.")